In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
#資料讀入以及合併
ls3 = pd.read_csv("C:\\Users\\user\\Documents\\GitHub\\big_data_competiton\\train.csv",encoding="BIG5")
ls3
repet = ls3[ls3["Y1"]=="Y"]
ls3 = pd.concat([ls3,repet,repet,repet,repet,repet,repet])# 
all_classes = ls3['Y1'].values
i = 0
while i < len(all_classes):
    if all_classes[i] == "Y":
        all_classes[i] = 1.0
    else:
        all_classes[i] = 0.0
    i +=1
ls3=ls3.drop("Y1",axis=1)

ls2 = pd.read_csv("C:\\Users\\user\\Documents\\GitHub\\big_data_competiton\\test.csv",encoding="BIG5")
name = ls2["CUS_ID"]
ls = pd.concat([ls3,ls2])  #最後調整好的是ls 

In [4]:
#資料預處理part1
need_to_0 = ["DIEBENEFIT_AMT","DIEACCIDENT_AMT","POLICY_VALUE_AMT","ANNUITY_AMT","EXPIRATION_AMT","ACCIDENT_HOSPITAL_REC_AMT",
        "DISEASES_HOSPITAL_REC_AMT","OUTPATIENT_SURGERY_AMT","INPATIENT_SURGERY_AMT","PAY_LIMIT_MED_MISC_AMT","FIRST_CANCER_AMT",
        "ILL_ACCELERATION_AMT","ILL_ADDITIONAL_AMT","LONG_TERM_CARE_AMT","MONTHLY_CARE_AMT",
                                 
            ]

need_to_N = ["IF_ISSUE_INSD_A_IND","IF_ISSUE_INSD_B_IND","IF_ISSUE_INSD_C_IND","IF_ISSUE_INSD_D_IND","IF_ISSUE_INSD_E_IND",
        "IF_ISSUE_INSD_F_IND","IF_ISSUE_INSD_G_IND","IF_ISSUE_INSD_H_IND","IF_ISSUE_INSD_I_IND","IF_ISSUE_INSD_J_IND",
        "IF_ISSUE_INSD_K_IND","IF_ISSUE_INSD_L_IND","IF_ISSUE_INSD_M_IND","IF_ISSUE_INSD_N_IND","IF_ISSUE_INSD_O_IND",
        "IF_ISSUE_INSD_P_IND","IF_ISSUE_INSD_Q_IND",
        "IF_ADD_INSD_F_IND","IF_ADD_INSD_L_IND","IF_ADD_INSD_Q_IND","IF_ADD_INSD_G_IND","IF_ADD_INSD_R_IND",    
        "IF_ADD_INSD_IND",
        "FINANCETOOLS_A","FINANCETOOLS_B","FINANCETOOLS_C","FINANCETOOLS_D","FINANCETOOLS_E","FINANCETOOLS_F","FINANCETOOLS_G",
        "X_A_IND","X_B_IND","X_C_IND","X_D_IND","X_E_IND","X_F_IND","X_G_IND","X_H_IND",
            'A_IND', 'B_IND', 'C_IND']
for i in need_to_0 :
    ls[i][ls[i].isna()] = 0
for i in need_to_N :
    ls[i][ls[i].isna()] = "N"
ls["GENDER"][ls["GENDER"].isna()] = "F"#性別 填女
ls["OCCUPATION_CLASS_CD"][ls["OCCUPATION_CLASS_CD"].isna()] = 1# 職業風險分數 眾數
ls["TERMINATION_RATE"][ls["TERMINATION_RATE"].isna()] = 0# 曾經解保單比率 眾數為0 加上值觀上也算是沒有解保單過
ls["LEVEL"][ls["LEVEL"].isna()] = 0  #往來等級 1~5 但沒有往來 所以決定填0
ls["RFM_M_LEVEL"][ls["RFM_M_LEVEL"].isna()] = 1#曾投保主約級數 3~10 無投保 推估1
ls["APC_1ST_AGE"][ls["APC_1ST_AGE"].isna()] = "無"        #第一次擔任被保人年齡(等級)
ls["RFM_R"][ls["RFM_R"].isna()] = "無"                    #上次要保人身分投保  (等級)這三個獨立出來當一個類別
ls["REBUY_TIMES_CNT"][ls["REBUY_TIMES_CNT"].isna()] = "無"#再次購買           (等級)
ls["GENDER"][ls["GENDER"].isna()] = "F"

#這段是對教育程度和年收入的交叉填值
new = ls[["EDUCATION_CD","ANNUAL_INCOME_AMT"]]
a = new.groupby("EDUCATION_CD").mean()
b = new.groupby("EDUCATION_CD").mean().unstack().unstack()
new["ANNUAL_INCOME_AMT"][(new["ANNUAL_INCOME_AMT"].isna()) & (new["EDUCATION_CD"]==1)]=int(b[1])
new["ANNUAL_INCOME_AMT"][(new["ANNUAL_INCOME_AMT"].isna()) & (new["EDUCATION_CD"]==2)]=int(b[2])
new["ANNUAL_INCOME_AMT"][(new["ANNUAL_INCOME_AMT"].isna()) & (new["EDUCATION_CD"]==3)]=int(b[3])
new["ANNUAL_INCOME_AMT"][(new["ANNUAL_INCOME_AMT"].isna()) & (new["EDUCATION_CD"]==4)]=int(b[4])
new["ANNUAL_INCOME_AMT"][(new["ANNUAL_INCOME_AMT"].isna())] = 0.000613 #甚麼都沒有的就填平均
new["EDUCATION_CD"][(new["EDUCATION_CD"].isna()) & (new["ANNUAL_INCOME_AMT"]<=0.000354)] = 1.0
new["EDUCATION_CD"][(new["EDUCATION_CD"].isna()) & (new["ANNUAL_INCOME_AMT"]<=0.000541)] = 2.0
new["EDUCATION_CD"][(new["EDUCATION_CD"].isna()) & (new["ANNUAL_INCOME_AMT"]<=0.000627)] = 3.0
new["EDUCATION_CD"][(new["EDUCATION_CD"].isna()) & (new["ANNUAL_INCOME_AMT"]<=1)] = 4.0
ls["EDUCATION_CD"] = new["EDUCATION_CD"]
ls["ANNUAL_INCOME_AMT"] = new["ANNUAL_INCOME_AMT"]

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [7]:
#這段是對教育程度和年收入的交叉填值
new = ls[["EDUCATION_CD","ANNUAL_INCOME_AMT"]]
a = new.groupby("EDUCATION_CD").mean()
b = new.groupby("EDUCATION_CD").mean().unstack().unstack()

#new["ANNUAL_INCOME_AMT"][(new["ANNUAL_INCOME_AMT"].isna()) & (new["EDUCATION_CD"]==1)]=int(b[1])
#new["ANNUAL_INCOME_AMT"][(new["ANNUAL_INCOME_AMT"].isna()) & (new["EDUCATION_CD"]==2)]=int(b[2])
#new["ANNUAL_INCOME_AMT"][(new["ANNUAL_INCOME_AMT"].isna()) & (new["EDUCATION_CD"]==3)]=int(b[3])
#new["ANNUAL_INCOME_AMT"][(new["ANNUAL_INCOME_AMT"].isna()) & (new["EDUCATION_CD"]==4)]=int(b[4])
#new["ANNUAL_INCOME_AMT"][(new["ANNUAL_INCOME_AMT"].isna())] = 0.000613 #甚麼都沒有的就填平均
#new["EDUCATION_CD"][(new["EDUCATION_CD"].isna()) & (new["ANNUAL_INCOME_AMT"]<=0.000354)] = 1.0
#new["EDUCATION_CD"][(new["EDUCATION_CD"].isna()) & (new["ANNUAL_INCOME_AMT"]<=0.000541)] = 2.0
#new["EDUCATION_CD"][(new["EDUCATION_CD"].isna()) & (new["ANNUAL_INCOME_AMT"]<=0.000627)] = 3.0
#new["EDUCATION_CD"][(new["EDUCATION_CD"].isna()) & (new["ANNUAL_INCOME_AMT"]<=1)] = 4.0
#ls["EDUCATION_CD"] = new["EDUCATION_CD"]
#ls["ANNUAL_INCOME_AMT"] = new["ANNUAL_INCOME_AMT"]

0


In [ ]:
#資料處理
check=ls.isna()
column =ls.columns.values.tolist()
k = 0

for i in column:
    if sum(check[i]) >0:  #將缺失值超過500個的column刪除
        #20000:74
        #15000"73
        #10000:73
        #5000:72
        #2000:71
        #500:70
        #100:67
        #1:59
        ls = ls.drop(i,axis=1)#丟掉有na的
"""
目前在苗鈺電腦上丟掉下列7個(尚未和高整合)
MARRIAGE_CD
INSD_1ST_AGE
APC_1ST_YEARDIF
ANNUAL_PREMIUM_AMT
L1YR_C_CNT
INSD_LAST_YEARDIF_CNT
BMI
"""
"""
預計真的要丟掉的有四個(和高整合後應該會丟掉的)(幫我檢查一下是不是這四個要丟掉，我自己忘了....)
INSD_1ST_AGE
APC_1ST_YEARDIF
L1YR_C_CNT
INSD_LAST_YEARDIF_CNT



"""
        
        print(i)
        k += 1
ls=ls.fillna(0) #將其餘缺失值以0補上

ls=ls.replace("無",-1)
ls=ls.replace("低",0)#將可量化的值取代
ls=ls.replace("中",1)
ls=ls.replace("中高",2)
ls=ls.replace("高",3)

ls = pd.get_dummies(ls)# 將剩餘不可量化、單純為類別的變數轉換為1000、0100的格式
#ls=ls.drop("Y1_N",axis=1)#上一步驟的轉換會多轉一個沒有意義的column 將這個刪掉
ls=ls.drop("CUS_ID",axis=1)#流水號刪除
b = ls.columns.values.tolist()#b=所有column的名子的串列
print(k)